In [4]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table
import base64
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
from CDLIB import AnimalShelter

###########################
# Data Manipulation / Model
###########################
username = "accuser"
password = "CSrules"
shelter = AnimalShelter(username, password)

df = pd.DataFrame.from_records(shelter.read_all({}))

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('Project2Submission')

image_filename = 'GraziosoSalvareLogo.png' 
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Center(html.B(html.H1('Creator: Christian Drouin'))),
    html.Center(html.Img(id='customer-image',
                         height='100',
                         width='100',
                         src='data:image/png;base64,{}'.format(encoded_image.decode()),
                         alt='customer image')),
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Hr(),
    
    #Radio buttons with rescue types rough shape
    html.Hr(),
    html.Div(
        dcc.RadioItems(
            id = 'filter_type' , 
            options = [
                {'label': 'Water Rescue', 'value': 'A'}, 
                {'label': 'Mountain Rescue', 'value': 'B'},
                {'label': 'Disaster Rescue', 'value': 'C'},
                {'Reset': 'Reset', 'value': 'D'}
            ],
            value = 'MTL',
            labelStyle = {'display': 'inline-block'}
        )
    ),
    
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
    ),
       
#This sets up the dashboard so that the chart and geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################

#Button callback rough shape
#Select option from Radiobuttons(input) and then update the dashboard with just those results(output)
@app.callback(Output('datatable-id', 'data'), #Output updated data from def below
              [Input('filter_type', 'value')]) 
#Add queries
def update_dashboard(filter_type):
        
        if(filter_type == 'A'):
            cursor = shelter.read_all({'breed':'Labrador Retriever Mix' 'Chesapeake Bay Retriever' 'Newfoundland',
                            'sex_upon_outcome': 'Intact Female', 'age_upon_outcome': [{"$gt":26},{"$lt":156}]}),
            df = pd.DataFrame.from_records(cursor)
        
        elif(filter_type == 'B'):
            cursor = shelter.read_all({'breed':'German Shepherd' 'Alaskan Malamute' 'Old English Sheepdog' 'Siberian Husky' 
                            'Rottweiler', 'sex_upon_outcome': 'Intact Male', 'age_upon_outcome': [{"$gt":26},{"$lt":156}]}),
            df = pd.DataFrame.from_records(cursor)
        
        elif(filter_type == 'C'):
            cursor = shelter.read_all({'breed':'Doberman Pinscher' 'German Shepherd' 'Golden Retriever' 'Bloodhound' 
                            'Rottweiler', 'sex_upon_outcome': 'Intact Male', 'age_upon_outcome': [{"$gt":20},{"$lt":300}]}),
            df = pd.DataFrame.from_records(cursor)
               
        elif(filter_type == 'D'):
            cursor = shelter.read_all({}),
            df = pd.DataFrame.from_records(cursor)
            
        else:
            cursor = shelter.read_all({}),
            df = pd.DataFrame.from_records(cursor)
        return df.to_dict('records')    

@app.callback([Output('datatable-id', 'style_data_conditional')],
              [Input('datatable-id', 'selected_columns')])
                  
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback([Output('graph-id', "children")], 
              [Input('datatable-id', "derived_viewport_data")])
                  
def update_graphs(viewData):
    # add code for chart of your choice (e.g. pie chart) #
    df = px.data.gapminder().query("filter_type")
    df.loc[df['pop'] < 2.e6, 'breed'] = 'filter type' 
    fig = px.pie(df, values='pop', names='breeds', title='Percentage of breed types')
    fig.show()

@app.callback([Output('map-id', "children")],
              [Input('datatable-id', "derived_viewport_data")])

def update_map(viewData):
    viewDF = pd.DataFrame.from_dict(viewData)
    dff = viewDF.loc[rows]
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[dff.loc[0,'location_lat'],dff.loc[0,'location_long']], children=[
                dl.Tooltip(dff['breed']),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[0,'name'])
                ])
            ])
        ])
    ]



app